In [1]:
import pandas as pd
import numpy as np
from pandarallel import pandarallel
import plotly.graph_objects as go

In [2]:
def week_day(param):
    return param.strftime("%A")

def weekOfyear(param):
    return param.strftime("%Y - %W")

In [3]:
df1 = pd.read_parquet('datasets/taxi_trip/8_months_taxi.parquet')

In [4]:
df = df1[['tpep_pickup_datetime']].copy()
del df1

In [5]:
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

In [6]:
#df['WeekDay'] = df.tpep_pickup_datetime
#df['YearWeek'] = df.tpep_pickup_datetime

In [7]:
#pandarallel.initialize(progress_bar=True)
#
#df.WeekDay = df.WeekDay.parallel_apply(week_day)
#df.YearWeek = df.YearWeek.parallel_apply(weekOfyear)

In [8]:
#------------------------CELDA AGREGADA---------------------
pandarallel.initialize()
df['WeekDay'] = df.tpep_pickup_datetime.parallel_apply(week_day)
df['YearWeek'] = df.tpep_pickup_datetime.parallel_apply(weekOfyear)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [9]:
z = pd.DataFrame((df.groupby(['WeekDay','YearWeek']).size()))
statistics = pd.DataFrame(z.values, columns=['viajes_amount'])

In [10]:
# get index from statistics
day = list(range(0,z.values.shape[0]))
for idx, i in enumerate(z.index):
    day[idx] = i[0]

statistics['DayWeek'] = day

In [13]:
#---------------------------MULTIPLICAR POR 8 PARA COMPENZAR LA CANTIDAD DEL SAMPLE--------------
f = statistics.DayWeek.unique()

boxes = list(range(0,7))
h = f[[3, 1, 5, 6, 4, 0, 2]]

for i in boxes:
    boxes[i] = np.array(statistics.viajes_amount[statistics.DayWeek == h[i]]*8)

In [14]:
fig = go.Figure()
for idx, i in enumerate(boxes):
    fig.add_trace(go.Box(y=i, name=h[idx],
                marker_color = 'blue'))

fig.show()